# DATASCI W261: Machine Learning at Scale
## Assignment Week 2
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Submission: 

### HW2.0:

#### What is a race condition in the context of parallel computation? Give an example.
A race condition is when a section of code is executed by multiple processes, and the order in which the processes execute will impact the final result.

![Race condition example](race_conditions.png)
Source: https://en.wikipedia.org/wiki/Race_condition#Example

#### What is MapReduce?
MapReduce can refer to multiple concepts:
- **Programming model:** Processes are split into a "mapping" phase, and a "reducing" phase. In the map phase, a certain function is mapped on to each value in a data set, and then in the reduce phase, the result of the map phase is aggregated. 
- **Execution framework:** This framework coordinates running processes written with the above model in mind.
- **Software implementation:** MapReduce is the name of Google's proprietary implementation of this programming model, while Apache Hadoop is the open-source implementation.

#### How does it differ from Hadoop?
Hadoop is the open-source implementation of Google's MapReduce. Hadoop consists of two parts: distributed storage of data, and distributed processing of data. HDFS is the storage part, and MapReduce is the processing part.

#### Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.
Hadoop is based on the MapReduce paradigm. The classic example of the MapReduce programming paradigm is word count. In the map phase of word count, each word in a document is assigned a count of 1. In the reduce phase, the counts for each unique word are summed to yield the final count of each word.

In [20]:
def hw2_0():
    doc = "Hello this is a test to test word count test should have a count of three".lower()
    key_vals = []
    
    print "MAP PHASE"
    for word in doc.split():
        print [word, 1]
        key_vals.append([word, 1])
    
    print "\nREDUCE PHASE"
    key_vals = sorted(key_vals)
    
    current_word = None
    current_count = 0
    
    for pair in key_vals:
        if current_word == pair[0]:
            print [current_word, current_count], "(intermediate step)"
            current_count += pair[1]
        else:
            if current_word:
                print [current_word, current_count], "FINAL SUM"
            current_word = pair[0]
            current_count = pair[1]

    print [current_word, current_count], "FINAL SUM"
    
hw2_0()

MAP PHASE
['hello', 1]
['this', 1]
['is', 1]
['a', 1]
['test', 1]
['to', 1]
['test', 1]
['word', 1]
['count', 1]
['test', 1]
['should', 1]
['have', 1]
['a', 1]
['count', 1]
['of', 1]
['three', 1]

REDUCE PHASE
['a', 1] (intermediate step)
['a', 2] FINAL SUM
['count', 1] (intermediate step)
['count', 2] FINAL SUM
['have', 1] FINAL SUM
['hello', 1] FINAL SUM
['is', 1] FINAL SUM
['of', 1] FINAL SUM
['should', 1] FINAL SUM
['test', 1] (intermediate step)
['test', 2] (intermediate step)
['test', 3] FINAL SUM
['this', 1] FINAL SUM
['three', 1] FINAL SUM
['to', 1] FINAL SUM
['word', 1] FINAL SUM


### HW2.1: Sort in Hadoop MapReduce
**Given as input: Records of the form `<integer, “NA”>`, where integer is any integer, and “NA” is just the empty string.**<br>
**Output: Sorted key value pairs of the form `<integer, “NA”>`; what happens if you have multiple reducers? Do you need additional steps? Explain.**

If there are multiple reducers, then a straightforward MapReduce process will yield outputs that are sorted within each reducer, but not sorted across all reducers. In order to output a sort across all reducers, an extra step needs to be implemented that will intelligently send keys to reducers so that the result from all reducers will yield a complete sort. For example, let's say our keys ranged from 0-300. If we had 3 reducers, we could send all keys in the range [0,100) to reducer 1, [100, 200) to reducer 2, and [200, 300] to reducer 3. Thus, the output of each reducer will yield documents that are completely sorted. We would need to balance the keys sent to each reducer to ensure that the load is still balanced between all reducers, which will require some calculations.

#### Write code to generate N  random records of the form `<integer, “NA”>`. Let N = 10,000.

We are going to need the Hadoop Streaming jar file, so let's download it here so that we know which one to use

In [22]:
!wget http://central.maven.org/maven2/org/apache/hadoop/hadoop-streaming/2.7.1/hadoop-streaming-2.7.1.jar

--2016-01-21 18:37:45--  http://central.maven.org/maven2/org/apache/hadoop/hadoop-streaming/2.7.1/hadoop-streaming-2.7.1.jar
Resolving central.maven.org... 23.235.47.209
Connecting to central.maven.org|23.235.47.209|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105736 (103K) [application/java-archive]
Saving to: “hadoop-streaming-2.7.1.jar”

100%[======================================>] 105,736     --.-K/s   in 0.1s    

2016-01-21 18:38:00 (1.06 MB/s) - “hadoop-streaming-2.7.1.jar” saved [105736/105736]



In [1]:
import random

with open("random.txt", "w") as myfile:
    for i in range(10000):
        myfile.write("{:d},{:s}\n".format(random.randint(0, 100000), "NA"))

#### Write the Python Hadoop streaming map-reduce job to perform this sort.

In [44]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Miki Seltzer
## Description: mapper code for HW2.1

import sys

# Our input comes from STDIN (standard input)
for line in sys.stdin:
    # In this case, we do not need to map the input to anything
    key, value = line.strip().split(',')
    print "%010d\t%s" % (int(key), value)

Overwriting mapper.py


In [45]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Miki Seltzer
## Description: reducer code for HW2.1

from operator import itemgetter
import sys

for line in sys.stdin:
    # In this case, we do not need to reduce anything
    key, value = line.strip().split('\t')
    print "%d\t%s" % (int(key), value)

Overwriting reducer.py


In [1]:
# Make HDFS directory and put random.txt there
!hdfs dfs -mkdir /user/miki/week02
!hdfs dfs -put random.txt /user/miki/week02

In [49]:
# Change permissions on mapper and reducer
!chmod +x mapper.py
!chmod +x reducer.py

# If output folder already exists, delete it
!hdfs dfs -rm -r /user/miki/week02/hw2_1_output

# Run job
!hadoop jar hadoop-streaming-2.7.1.jar \
-D mapred.reduce.tasks=4 \
-mapper /home/cloudera/Documents/W261-Fall2016/Week02/mapper.py \
-reducer /home/cloudera/Documents/W261-Fall2016/Week02/reducer.py \
-input /user/miki/week02/random.txt \
-output /user/miki/week02/hw2_1_output

16/01/22 12:59:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/miki/week02/hw2_1_output
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.5.0.jar] /tmp/streamjob4129142604574616452.jar tmpDir=null
16/01/22 12:59:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/22 12:59:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/22 12:59:17 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/22 12:59:17 INFO mapreduce.JobSubmitter: number of splits:2
16/01/22 12:59:17 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/22 12:59:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453405632837_0022
16/01/22 12:59:18 INFO impl.YarnClientImpl: Submitted application application_1453405632837_0022
16/01/22 12:59:18 INFO mapreduce.Job: The url to track the job: http://quick

In [52]:
!hdfs dfs -cat /user/miki/week02/hw2_1_output/part-00000

### HW2.2: Using the Enron data from HW1 and Hadoop MapReduce streaming, write mapper/reducer pair that  will determine the number of occurrences of a single, user-specified word. 

Examine the word “assistance” and report your results. To do so, make sure that mapper.py counts all occurrences of a single word, and reducer.py collates the counts of the single word.

#### Load enronemail_1h.txt into HDFS

In [47]:
!hdfs dfs -put enronemail_1h.txt /user/miki/week02

In [93]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Miki Seltzer
## Description: mapper code for HW2.2

import sys
import string
import re

# Keyword from user input
keyword = sys.argv[1]

# Our input comes from STDIN (standard input)
for line in sys.stdin:
    
    # Strip white space from line, then split into fields
    # Replace commas with spaces (we are using commas as a delimiter as well)
    # Remove remaining punctuation from subject and body
    # Concatenate, then split subject and body by spaces
    # Some records are malformed -- if there is a 4th field, use it
    fields = line.strip().split('\t')
    subj = fields[2].replace(',', ' ')
    subj = subj.translate(string.maketrans("",""), string.punctuation)
    if len(fields) == 4:
        body = fields[3].replace(',', ' ')
        body = body.translate(string.maketrans("",""), string.punctuation)
    else:
        body = ""
    words = subj + " " + body
    words = words.split()
    
    # Loop through words
    # If word is keyword, write to file
    # key = word
    # value = 1
    for word in words:
        if keyword == word:
            print "%s\t%s" % (word, 1)

Overwriting mapper.py


In [94]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Miki Seltzer
## Description: reducer code for HW2.2

from operator import itemgetter
import sys

# Initialize some variables
# We know that the words will be sorted
# We need to keep track of state
prev_word = None
prev_count = 0
word = None

for line in sys.stdin:
    # Strip and split line from mapper
    word, count = line.strip().split('\t', 1)
    
    # If possible, turn count into an int (it's read as a string)
    try:
        count = int(count)
    except ValueError:
        # We couldn't make count into an int, so move on
        continue
        
    # Since the words will be sorted, all counts for a word will be grouped
    if prev_word == word:
        # If prev_word is word, then we haven't changed words
        # Just update prev_count
        prev_count += count
    else:
        # We've encountered a new word!
        # This might be the first word, though
        if prev_word:
            # We need to print the last word we were on
            print "%s\t%s" % (prev_word, prev_count)
        
        # Now we need to initialize our variables for the new word and count
        prev_word = word
        prev_count = count

# We have reached the end of the file, so print the last word and count
if prev_word == word:
    print "%s\t%s" % (prev_word, prev_count)

Overwriting reducer.py


#### Run Hadoop streaming command

In [95]:
# Change permissions on mapper and reducer
!chmod +x mapper.py
!chmod +x reducer.py

# Delete output folder if it exists
!hdfs dfs -rm -r /user/miki/week02/hw2_2_output

# Run job
!hadoop jar hadoop-streaming-2.7.1.jar \
-mapper '/home/cloudera/Documents/W261-Fall2016/Week02/mapper.py assistance' \
-reducer /home/cloudera/Documents/W261-Fall2016/Week02/reducer.py \
-input /user/miki/week02/enronemail_1h.txt \
-output /user/miki/week02/hw2_2_output

16/01/22 13:59:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/miki/week02/hw2_2_output
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.5.0.jar] /tmp/streamjob2515332195244567054.jar tmpDir=null
16/01/22 13:59:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/22 13:59:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/22 13:59:10 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/22 13:59:10 INFO mapreduce.JobSubmitter: number of splits:2
16/01/22 13:59:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453405632837_0027
16/01/22 13:59:11 INFO impl.YarnClientImpl: Submitted application application_1453405632837_0027
16/01/22 13:59:11 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1453405632837_0027/
16/01/22 13:59:11 INFO mapreduce.Job: Running job: job_1453405

#### Look at output of job

In [96]:
!hdfs dfs -cat /user/miki/week02/hw2_2_output/part-00000

assistance	10


### HW2.2.1: Using Hadoop MapReduce and your wordcount job (from HW2.2) determine the top-10 occurring tokens (most frequent tokens)

In [ ]:
##############################
#
#    DO THIS!!!!!!!
#
##############################

### HW2.3: Multinomial NAIVE BAYES with NO Smoothing
Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer job(s) that will both learn Naive Bayes classifier and classify the Enron email messages using the learnt Naive Bayes classifier. Use all white-space delimited tokens as independent input variables (assume spaces, fullstops, commas as delimiters). 

Note: for multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

$$
\frac{\text{number of times "assistance" occurs in SPAM labeled documents}}{\text{the number of words in documents labeled SPAM}}
$$

#### Mapper/Reducer for fitting NB

In [446]:
%%writefile mapper_fit.py
#!/usr/bin/python
## mapper_fit.py
## Author: Miki Seltzer
## Description: mapper code for HW2.3

import sys
import string
import re

counts = {
    '1':{
        '*words':0,
        '*docs':0
    },
    '0':{
        '*words':0,
        '*docs':0
    }
}
total_docs = 0

# Our input comes from STDIN (standard input)
for line in sys.stdin:
    
    # Strip white space from line, then split into fields
    # Replace commas with spaces (we are using commas as a delimiter as well)
    # Remove remaining punctuation from subject and body
    # Concatenate, then split subject and body by spaces
    # Some records are malformed -- if there is a 4th field, use it
    fields = line.strip().split('\t')
    
    # Keep track of document counts
    spam = fields[1]
    counts[spam]['*docs'] += 1
    total_docs += 1
    
    subj = fields[2].replace(',', ' ')
    subj = subj.translate(string.maketrans("",""), string.punctuation)
    if len(fields) == 4:
        body = fields[3].replace(',', ' ')
        body = body.translate(string.maketrans("",""), string.punctuation)
    else:
        body = ""
    words = subj + " " + body
    words = words.split()
    
    # Loop through words
    # If word is not trivial, write to file
    # key = word
    # value = 1
    for word in words:
        if len(word) > 0 and repr(word)[1] != '\\':
            print "%s^%s^%s" % (spam, word, 1)
            counts[spam]['*words'] += 1

# At the end, output document and word counts
for spam in counts:
    for item in counts[spam]:
        print "%s^%s^%s" % (spam, item, counts[spam][item])
    print "%s^%s^%s" % (spam, '*alldocs', total_docs)


Writing mapper_fit.py


In [447]:
%%writefile reducer_fit.py
#!/usr/bin/python
## reducer_fit.py
## Author: Miki Seltzer
## Description: reducer code for HW2.3

from operator import itemgetter
import sys
import math

# Initialize some variables
# We know that the words will be sorted
# We need to keep track of state
prev_word = None
prev_cat = None
prev_count = 0
cat_words = 0.0
cat_docs = 0.0
total_docs = 0.0
word = None
num_smoothing_errors = 0
num_value_errors = 0

for line in sys.stdin:
    # Strip and split line from mapper
    line = line.replace('\n', '')
    key, count = line.split('\t')
    cat, word = key.split('^')
    
    # If possible, turn count into an int (it's read as a string)
    try:
        count = int(count)
    except ValueError:
        # We couldn't make count into an int, so move on
        num_value_errors += 1
        continue
    
    # Primary key from mapper was category
    # We are going to get all words in one category
    # Mapper output should be sorted by category, then word
    if prev_cat != cat:
        # We have changed categories, so update prev_cat
        prev_cat = cat
        
    if prev_word == word:
        # We have not moved to a new word
        # Just update the count of this category
        prev_count += count
        
    else:
        # We have encountered a new word!
        # If this is the first word, we don't need to print anything
        if prev_word: 
            # This is not the first word.
            # If prev_word is a special word, set vars appropriately
            # Else this is a normal word, and we can calculate P(word|cat)
            if prev_word == '*alldocs': 
                # We have the full count for total_docs
                total_docs = float(prev_count)
            elif prev_word == '*docs': 
                # We should now have the data to calculate our prior
                cat_docs = float(prev_count)
                print '%s\t%s\t%s' % ('*prior', 
                                      prev_cat, 
                                      math.log(cat_docs/total_docs))
            elif prev_word == '*words': 
                # This allows us to calculate P(word|cat)
                cat_words = float(prev_count)
            else:
                # The prev_word was normal
                if prev_count > 0:
                    # The prev_count must be > 0 to write
                    print '%s\t%s\t%s' % (prev_word, 
                                          prev_cat, 
                                          math.log(prev_count/cat_words))
                else:
                    # The prev_count was not > 0, this will cause a log error
                    num_smoothing_errors += 1
            
        # Now we need to initialize our variables
        prev_word = word
        prev_count = count

# We've reached the end of the file
# Print the last word and count
if prev_count > 0:
    # The prev_count must be > 0 to write
    print '%s\t%s\t%s' % (prev_word, 
                          prev_cat, 
                          math.log(prev_count/cat_words))
else:
    # The prev_count was not > 0, this will cause a log error
    num_smoothing_errors += 1  

# Print the number of smoothing errors
print '%s\t%s\t%s' % ('*smoothing_errors', 
                      'all_categories', 
                      num_smoothing_errors)

Writing reducer_fit.py


In [453]:
# Change permissions on mapper and reducer
!chmod +x mapper_fit.py
!chmod +x reducer_fit.py

# Delete output folder if it exists
!hdfs dfs -rm -r /user/miki/week02/hw2_3_output_fit

# Run job
!hadoop jar hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator=^ \
-D stream.num.map.output.key.fields=2 \
-D mapreduce.map.output.key.field.separator=^ \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-input /user/miki/week02/enronemail_1h.txt \
-output /user/miki/week02/hw2_3_output_fit \
-mapper /home/cloudera/Documents/W261-Fall2016/Week02/mapper_fit.py \
-reducer /home/cloudera/Documents/W261-Fall2016/Week02/reducer_fit.py 


16/01/22 23:44:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/miki/week02/hw2_3_output_fit
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.5.0.jar] /tmp/streamjob6772481191612508411.jar tmpDir=null
16/01/22 23:44:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/22 23:44:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/22 23:44:28 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/22 23:44:28 INFO mapreduce.JobSubmitter: number of splits:2
16/01/22 23:44:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453405632837_0125
16/01/22 23:44:29 INFO impl.YarnClientImpl: Submitted application application_1453405632837_0125
16/01/22 23:44:29 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1453405632837_0125/
16/01/22 23:44:29 INFO mapreduce.Job: Running job: job_145

In [450]:
#!hdfs dfs -cat /user/miki/week02/hw2_3_output_fit/part-00000

In [458]:
!hdfs dfs -ls /user/miki/week02/hw2_3_output_fit/

Found 2 items
-rw-r--r--   1 cloudera supergroup          0 2016-01-22 23:44 /user/miki/week02/hw2_3_output_fit/_SUCCESS
-rw-r--r--   1 cloudera supergroup     165713 2016-01-22 23:44 /user/miki/week02/hw2_3_output_fit/part-00000


#### Mapper/Reducer for predicting NB

In [471]:
%%writefile mapper_predict.py
#!/usr/bin/python
## mapper_predict.py
## Author: Miki Seltzer
## Description: mapper code for HW2.3

import sys
import string
import re

# Our input comes from STDIN (standard input)
for line in sys.stdin:
    
    # Strip white space from line, then split into fields
    fields = line.strip().split('\t')
    
    # Look at the first field, and see if it matches the doc_id pattern
    pattern = re.compile('^\d{4}.\d{4}-\d{2}-\d{2}\.\w+')
    if pattern.match(fields[0]):
        # We know this is an email
        # Replace commas with spaces (we are using commas as a delimiter as well)
        # Remove remaining punctuation from subject and body
        # Concatenate, then split subject and body by spaces
        # Some records are malformed -- if there is a 4th field, use it

        print "email:", fields
#         spam = fields[1]
#         subj = fields[2].replace(',', ' ')
#         subj = subj.translate(string.maketrans("",""), string.punctuation)
#         if len(fields) == 4:
#             body = fields[3].replace(',', ' ')
#             body = body.translate(string.maketrans("",""), string.punctuation)
#         else:
#             body = ""
#         words = subj + " " + body
#         words = words.split()

#         # Loop through words
#         # If word is not trivial, write to file
#         # key = word
#         # value = 1
#         for word in words:
#             if len(word) > 0 and repr(word)[1] != '\\':
#                 print "%s^%s^%s" % (spam, word, 1)

    else:
        print "not email:", fields
    

Overwriting mapper_predict.py


In [455]:
%%writefile reducer_predict.py
#!/usr/bin/python
## reducer_predict.py
## Author: Miki Seltzer
## Description: reducer code for HW2.3

from operator import itemgetter
import sys
import string

# Our input comes from STDIN (standard input)
for line in sys.stdin:
    print line

Writing reducer_predict.py


In [472]:
# Change permissions on mapper and reducer
!chmod +x mapper_predict.py
!chmod +x reducer_predict.py

# Delete output folder if it exists
!hdfs dfs -rm -r /user/miki/week02/hw2_3_output_predict

# Run job
!hadoop jar hadoop-streaming-2.7.1.jar \
-input /user/miki/week02/hw2_3_output_fit/part* \
-input /user/miki/week02/enronemail_1h.txt \
-output /user/miki/week02/hw2_3_output_predict \
-mapper /home/cloudera/Documents/W261-Fall2016/Week02/mapper_predict.py \
-reducer /home/cloudera/Documents/W261-Fall2016/Week02/reducer_predict.py 


16/01/23 00:18:51 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/miki/week02/hw2_3_output_predict
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.5.0.jar] /tmp/streamjob8679788927932437756.jar tmpDir=null
16/01/23 00:18:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/23 00:18:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/23 00:18:55 INFO mapred.FileInputFormat: Total input paths to process : 2
16/01/23 00:18:55 INFO mapreduce.JobSubmitter: number of splits:3
16/01/23 00:18:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453405632837_0132
16/01/23 00:18:56 INFO impl.YarnClientImpl: Submitted application application_1453405632837_0132
16/01/23 00:18:56 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1453405632837_0132/
16/01/23 00:18:56 INFO mapreduce.Job: Running job: job

In [475]:
!hdfs dfs -cat /user/miki/week02/hw2_3_output_predict/part-00000

email: ['0001.1999-12-10.farmer', '0', ' christmas tree farm pictures', 'NA']	
	
email: ['0001.1999-12-10.kaminski', '0', ' re: rankings', ' thank you.']	
	
email: ['0001.2000-01-17.beck', '0', ' leadership development pilot', ' sally:  what timing, ask and you shall receive. as per our discussion, listed below  is an update on the leadership pilot. your vendor selection team will  receive an update and even more information later in the week.  on the lunch & learn for energy operations, the audience and focus will be  your group. we are ready to start up when appropriate.  thank you for your time today. please call me if you have any questions at  x 33597.  ----------------------forwarded by julie armstrong/corp/enron on 01/17/2000  06:44 pm---------------------------  from: susan runkel @ ect 01/17/2000 03:22 pm  to: cindy skinner/hou/ect @ ect, brad mcsherry/hou/ect @ ect, norma  villarreal/hou/ect @ ect, kimberly rizzi/hou/ect @ ect, fran l mayes/hou/ect @ ect,  gary buck/hou/ect @